In [2]:
import os 
import json
import pandas as pd
import traceback
from langchain_together import Together


# Load the .env file variable


In [3]:
from langchain.chat_models import ChatOpenAI

In [4]:
# Load the environment file (containing the api keys)
from dotenv import load_dotenv

load_dotenv() 

True

In [5]:
KEY = os.getenv("OPENAI_API_KEY")

In [6]:
# Using the langchain class to get llm
llm = ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.5)

/opt/anaconda3/envs/mk-llm/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [7]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2


In [8]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    }
}

In [9]:
TEMPLATE = """
Text: {text}
You are an expert MCQ maker, Given the above text, it is your job to \
create a quiz pf {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}"""

In [10]:
# Creating prompt template
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number","subject", "tone", "response_json"],
    template= TEMPLATE
)

In [11]:
# LLMChain: Chain to run queries against the llm
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key='quiz', verbose=True)

NameError: name 'llm' is not defined

In [12]:
TEMPLATE2 ="""
You are an expert english grammarian and writer. Given a multiple choice quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis
if the quiz is not at par with the cognitive and analytical abilities of the students, \
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the students
Quiz_MCQs:
{quiz}

Check from an expert English writer of the above quiz:
"""

In [1]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

NameError: name 'PromptTemplate' is not defined

In [ ]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [ ]:
# Connecting two chain together, sequential chain

generate_evaluate_chain = SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject","quiz", "tone", "response_json"], output_variables=["quiz","review"], verbose=True)

In [2]:
file_path = "../data.txt"

In [3]:
file_path

'../data.txt'

In [4]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [5]:
print(TEXT)

Artificial intelligence

Machine learning as subfield of AI[22]
As a scientific endeavor, machine learning grew out of the quest for artificial intelligence (AI). In the early days of AI as an academic discipline, some researchers were interested in having machines learn from data. They attempted to approach the problem with various symbolic methods, as well as what were then termed "neural networks"; these were mostly perceptrons and other models that were later found to be reinventions of the generalized linear models of statistics.[23] Probabilistic reasoning was also employed, especially in automated medical diagnosis.[24]: 488 

However, an increasing emphasis on the logical, knowledge-based approach caused a rift between AI and machine learning. Probabilistic systems were plagued by theoretical and practical problems of data acquisition and representation.[24]: 488  By 1980, expert systems had come to dominate AI, and statistics was out of favor.[25] Work on symbolic/knowledge-ba

In [12]:
# Serialize the python dictionary into a JSON- formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [13]:
NUMBER = 5
SUBJECT = "Machine Learning"
TONE ="Normal"

In [ ]:
# Setting up token usage tracking 
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )

In [14]:
print(f"Total tokens: {cb.total_tokens}")
print(f"Prompt token: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

NameError: name 'cb' is not defined

In [15]:
response

NameError: name 'response' is not defined

In [ ]:
quiz = response.get("quiz")

In [ ]:
json.loads(quiz)

In [ ]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options ="|".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [ ]:
quiz_table_data

In [ ]:
pd.DataFrame(quiz_table_data)

In [ ]:
quiz.to_csv("machinelearning.csv", index=False)